In [1]:
import os, sys, random
import xml.etree.ElementTree as ET
import mxnet as mx
import cv2
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
import random
from skimage import io
from shutil import copyfile
import sys
import time

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
# training annotation file
training_annotation_loc = r'D:\uc file\Intern ship\projects\data\OIDv4_ToolKit\OID\csv_folder\train-annotations-bbox.csv'
training_annotation_file = pd.read_csv(training_annotation_loc)
print("records present for training : ",training_annotation_file.shape)
training_annotation_file.head()

records present for training :  (14610229, 13)


,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
0,000002b66c9c498e,xclick,/m/01g317,1,0.012500,0.195312,0.148438,0.587500,0,1,0,0,0
1,000002b66c9c498e,xclick,/m/01g317,1,0.025000,0.276563,0.714063,0.948438,0,1,0,0,0
2,000002b66c9c498e,xclick,/m/01g317,1,0.151562,0.310937,0.198437,0.590625,1,0,0,0,0
3,000002b66c9c498e,xclick,/m/01g317,1,0.256250,0.429688,0.651563,0.925000,1,0,0,0,0
4,000002b66c9c498e,xclick,/m/01g317,1,0.257812,0.346875,0.235938,0.385938,1,0,0,0,0


In [3]:
# The class-descriptions-boxable.csv contains the name of all 600 classes with their corresponding ‘LabelName’
class_descriptions_file=r'D:\uc file\Intern ship\projects\data\OIDv4_ToolKit\OID\csv_folder\class-descriptions-boxable.csv'
class_descriptions = pd.read_csv(class_descriptions_file, header=None)
print(class_descriptions.shape)
class_descriptions.head()

(601, 2)


,0,1
0,/m/011k07,Tortoise
1,/m/011q46kg,Container
2,/m/012074,Magpie
3,/m/0120dh,Sea turtle
4,/m/01226z,Football


In [4]:
# Find the label_name for 'Apple', 'Orange' and 'Light Switches' classes
Laptop = class_descriptions[class_descriptions[1]=='Laptop']
Mobile_phone = class_descriptions[class_descriptions[1]=='Mobile phone']

Laptop


label_name_Laptop = Laptop[0].values[0]
label_name_Mobile_phone = Mobile_phone[0].values[0]

In [5]:
Mobile_phone

,0,1
312,/m/050k8,Mobile phone


In [6]:
Laptop

,0,1
54,/m/01c648,Laptop


In [7]:
Laptop_bbox = training_annotation_file[training_annotation_file['LabelName']==label_name_Laptop]
Mobile_phone_bbox = training_annotation_file[training_annotation_file['LabelName']==label_name_Mobile_phone]
Mobile_phone_bbox


,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
1149,0000b9115cdf1e54,xclick,/m/050k8,1,0.095703,0.707031,0.497396,0.949219,0,0,0,0,0
2487,00016982f6086d39,xclick,/m/050k8,1,0.000000,0.997222,0.000000,0.998438,0,1,0,0,0
3963,0002a1a755d730ca,xclick,/m/050k8,1,0.178125,0.409375,0.339583,0.695833,1,0,0,0,0
5408,000384bb6da4764b,xclick,/m/050k8,1,0.695545,0.985148,0.434375,0.803750,0,0,0,0,0
8040,0005829bfcf77ca3,xclick,/m/050k8,1,0.611875,0.699375,0.585458,0.787535,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14602894,ffd92391b7d03016,activemil,/m/050k8,1,0.798437,0.964063,0.392188,0.531250,0,1,0,0,0
14604511,ffe1e233465f0a49,activemil,/m/050k8,1,0.025602,0.944277,0.040493,0.917253,0,1,0,0,0
14609308,fffa17e6456b26d6,activemil,/m/050k8,1,0.224771,0.872477,0.139375,0.801250,1,0,0,0,0
14609730,fffcbc0b28c934f7,activemil,/m/050k8,1,0.078750,0.292500,0.176667,0.615833,0,0,0,1,0


In [8]:
print('There are %d apples in the dataset' %(len(Laptop_bbox)))
print('There are %d oranges in the dataset' %(len(Mobile_phone_bbox)))

There are 9327 apples in the dataset
There are 6365 oranges in the dataset


In [9]:
Laptop_id = Laptop_bbox['ImageID']
Mobile_phone_id = Mobile_phone_bbox['ImageID']
Laptop_id.count() # 627 different ids are present
Laptop_id  # these are the details of 627 different ids present in apple_img_id

583         000060e3121c7305
3539        00022a6311159428
4018        0002af997ecdfcf4
5473        000396ae942e8778
6449        00048edce2a37368
                  ...       
14602447    ffd6aa9288085a8f
14604500    ffe1bfb06dd60d55
14605390    ffe4ab4dc71ccaa3
14605393    ffe4ab4dc71ccaa3
14610134    ffff83c6a0e1c6c0
Name: ImageID, Length: 9327, dtype: object

In [10]:
concated_df=pd.concat([Laptop_bbox, Mobile_phone_bbox], axis=0)
concated_df

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
583,000060e3121c7305,xclick,/m/01c648,1,0.322500,0.389375,0.580131,0.669166,0,0,0,0,0
3539,00022a6311159428,xclick,/m/01c648,1,0.126875,0.484375,0.638281,0.871094,0,0,0,0,0
4018,0002af997ecdfcf4,xclick,/m/01c648,1,0.250000,0.363750,0.236667,0.334444,0,0,0,0,0
5473,000396ae942e8778,xclick,/m/01c648,1,0.000000,0.995000,0.000000,0.996390,0,0,0,0,0
6449,00048edce2a37368,xclick,/m/01c648,1,0.000000,0.225000,0.710000,0.999167,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14602894,ffd92391b7d03016,activemil,/m/050k8,1,0.798437,0.964063,0.392188,0.531250,0,1,0,0,0
14604511,ffe1e233465f0a49,activemil,/m/050k8,1,0.025602,0.944277,0.040493,0.917253,0,1,0,0,0
14609308,fffa17e6456b26d6,activemil,/m/050k8,1,0.224771,0.872477,0.139375,0.801250,1,0,0,0,0
14609730,fffcbc0b28c934f7,activemil,/m/050k8,1,0.078750,0.292500,0.176667,0.615833,0,0,0,1,0


In [11]:
concated_df['ClassName']=concated_df['LabelName']
concated_df['ClassName']

583         /m/01c648
3539        /m/01c648
4018        /m/01c648
5473        /m/01c648
6449        /m/01c648
              ...    
14602894     /m/050k8
14604511     /m/050k8
14609308     /m/050k8
14609730     /m/050k8
14609731     /m/050k8
Name: ClassName, Length: 15692, dtype: object

In [12]:
#classes = ['apple', 'orange', 'Light switch']
#labels = ['/m/014j1m', '/m/0cyhj_', '/m/03jbxj']


mapping = {
    '/m/01c648': 'Laptop',
    '/m/050k8': 'Mobile_phone',
}

concated_df['ClassName'] = [mapping[word] for word in concated_df['ClassName']]
concated_df['ClassName']

concated_df

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside,ClassName
583,000060e3121c7305,xclick,/m/01c648,1,0.322500,0.389375,0.580131,0.669166,0,0,0,0,0,Laptop
3539,00022a6311159428,xclick,/m/01c648,1,0.126875,0.484375,0.638281,0.871094,0,0,0,0,0,Laptop
4018,0002af997ecdfcf4,xclick,/m/01c648,1,0.250000,0.363750,0.236667,0.334444,0,0,0,0,0,Laptop
5473,000396ae942e8778,xclick,/m/01c648,1,0.000000,0.995000,0.000000,0.996390,0,0,0,0,0,Laptop
6449,00048edce2a37368,xclick,/m/01c648,1,0.000000,0.225000,0.710000,0.999167,1,1,0,0,0,Laptop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14602894,ffd92391b7d03016,activemil,/m/050k8,1,0.798437,0.964063,0.392188,0.531250,0,1,0,0,0,Mobile_phone
14604511,ffe1e233465f0a49,activemil,/m/050k8,1,0.025602,0.944277,0.040493,0.917253,0,1,0,0,0,Mobile_phone
14609308,fffa17e6456b26d6,activemil,/m/050k8,1,0.224771,0.872477,0.139375,0.801250,1,0,0,0,0,Mobile_phone
14609730,fffcbc0b28c934f7,activemil,/m/050k8,1,0.078750,0.292500,0.176667,0.615833,0,0,0,1,0,Mobile_phone


In [13]:
# Data format for faster R-Cnn   :    ['FileName', 'XMin', 'XMax', 'YMin', 'YMax', 'ClassName'])
train_df= concated_df.loc[:,['ImageID','XMin','XMax','YMin','YMax','ClassName']]
train_df

,ImageID,XMin,XMax,YMin,YMax,ClassName
583,000060e3121c7305,0.322500,0.389375,0.580131,0.669166,Laptop
3539,00022a6311159428,0.126875,0.484375,0.638281,0.871094,Laptop
4018,0002af997ecdfcf4,0.250000,0.363750,0.236667,0.334444,Laptop
5473,000396ae942e8778,0.000000,0.995000,0.000000,0.996390,Laptop
6449,00048edce2a37368,0.000000,0.225000,0.710000,0.999167,Laptop
...,...,...,...,...,...,...
14602894,ffd92391b7d03016,0.798437,0.964063,0.392188,0.531250,Mobile_phone
14604511,ffe1e233465f0a49,0.025602,0.944277,0.040493,0.917253,Mobile_phone
14609308,fffa17e6456b26d6,0.224771,0.872477,0.139375,0.801250,Mobile_phone
14609730,fffcbc0b28c934f7,0.078750,0.292500,0.176667,0.615833,Mobile_phone


In [14]:
train_df['ImageID']=train_df['ImageID']+'.jpg'
train_df

,ImageID,XMin,XMax,YMin,YMax,ClassName
583,000060e3121c7305.jpg,0.322500,0.389375,0.580131,0.669166,Laptop
3539,00022a6311159428.jpg,0.126875,0.484375,0.638281,0.871094,Laptop
4018,0002af997ecdfcf4.jpg,0.250000,0.363750,0.236667,0.334444,Laptop
5473,000396ae942e8778.jpg,0.000000,0.995000,0.000000,0.996390,Laptop
6449,00048edce2a37368.jpg,0.000000,0.225000,0.710000,0.999167,Laptop
...,...,...,...,...,...,...
14602894,ffd92391b7d03016.jpg,0.798437,0.964063,0.392188,0.531250,Mobile_phone
14604511,ffe1e233465f0a49.jpg,0.025602,0.944277,0.040493,0.917253,Mobile_phone
14609308,fffa17e6456b26d6.jpg,0.224771,0.872477,0.139375,0.801250,Mobile_phone
14609730,fffcbc0b28c934f7.jpg,0.078750,0.292500,0.176667,0.615833,Mobile_phone


In [15]:
train_df.shape
train_df.to_csv('train.csv')
train_df['ClassName'].value_counts()

Laptop          9327
Mobile_phone    6365
Name: ClassName, dtype: int64

In [40]:
# Writing train.csv to annotation.txt

train_df = pd.read_csv('train.csv')

# for training
with open("annotation.txt", "w+") as f:
  for idx, row in train_df.iterrows():
    img = cv2.imread('D:/uc file/Intern ship/projects/month/Fast_RCNN/datanew/' + row['ImageID'])
    if(img is not None):
        
#         print(img)  #none 
        height, width = img.shape[:2]
        x1 = int(row['XMin'] * width)
        x2 = int(row['XMax'] * width)
        y1 = int(row['YMin'] * height)
        y2 = int(row['YMax'] * height)
      
        train_file_path = 'D:/uc file/Intern ship/projects/month/Fast_RCNN/train'
        fileName = os.path.join(train_file_path, row['ImageID'])
        className = row['ClassName']
        f.write(fileName + ',' + str(x1) + ',' + str(y1) + ',' + str(x2) + ',' + str(y2) + ',' + className + '\n')

[[[181 177 189]
  [181 177 188]
  [180 176 187]
  ...
  [ 29 117 177]
  [ 28 116 176]
  [ 27 115 175]]

 [[181 177 189]
  [180 176 187]
  [180 177 186]
  ...
  [ 31 119 179]
  [ 30 118 178]
  [ 29 117 177]]

 [[180 176 187]
  [180 177 186]
  [178 177 186]
  ...
  [ 34 122 182]
  [ 32 120 180]
  [ 31 119 179]]

 ...

 [[171 162 172]
  [115 106 116]
  [ 78  69  79]
  ...
  [ 95 100 139]
  [104 108 149]
  [ 53  59 100]]

 [[184 175 185]
  [125 116 126]
  [ 84  75  85]
  ...
  [102 101 135]
  [119 117 153]
  [ 92  90 126]]

 [[189 180 190]
  [147 138 148]
  [ 92  83  93]
  ...
  [133 124 151]
  [117 105 133]
  [212 200 228]]]
[[[122  90  31]
  [115  84  33]
  [111  80  41]
  ...
  [ 21  79 114]
  [ 23  76 109]
  [ 28  79 112]]

 [[117  87  32]
  [112  82  33]
  [112  79  40]
  ...
  [ 18  78 114]
  [ 20  75 112]
  [ 23  76 109]]

 [[115  87  33]
  [115  84  33]
  [106  77  33]
  ...
  [ 20  78 113]
  [ 25  77 113]
  [ 26  78 108]]

 ...

 [[ 45  49  54]
  [ 48  53  56]
  [ 50  55  58]
  ..

[[[  0  67 120]
  [  3  70 119]
  [  0  72 128]
  ...
  [251 253 253]
  [251 253 253]
  [251 253 253]]

 [[  1  65 123]
  [  1  66 117]
  [  0  70 123]
  ...
  [250 252 252]
  [251 253 253]
  [251 253 253]]

 [[  2  67 122]
  [  0  69 119]
  [  1  67 118]
  ...
  [251 253 253]
  [251 253 253]
  [251 253 253]]

 ...

 [[  1 119 202]
  [  1 117 201]
  [  1 117 202]
  ...
  [  0   5  14]
  [  1   6  15]
  [  0   5  12]]

 [[  1 117 202]
  [  0 116 201]
  [  0 115 200]
  ...
  [  1   5  16]
  [  2   7  16]
  [  1   6  15]]

 [[  1 114 200]
  [  0 116 201]
  [  0 119 204]
  ...
  [  1   5  16]
  [  0   5  14]
  [  1   6  15]]]
[[[138 132 127]
  [136 130 125]
  [137 131 126]
  ...
  [121 120 116]
  [117 121 115]
  [117 125 115]]

 [[134 132 122]
  [134 131 123]
  [135 131 126]
  ...
  [125 121 120]
  [120 122 116]
  [119 121 115]]

 [[131 131 119]
  [133 132 122]
  [131 131 125]
  ...
  [126 121 120]
  [125 122 118]
  [123 120 116]]

 ...

 [[ 82  83  81]
  [ 85  83  82]
  [ 95  89  84]
  ..

[[[ 51  36  20]
  [ 49  37  19]
  [ 53  39  21]
  ...
  [239  48   0]
  [238  47   0]
  [238  47   0]]

 [[ 46  33  17]
  [ 50  35  19]
  [ 52  38  20]
  ...
  [242  48   0]
  [238  48   0]
  [238  47   0]]

 [[ 51  35  22]
  [ 51  37  19]
  [ 50  36  17]
  ...
  [239  49   0]
  [238  48   0]
  [238  48   0]]

 ...

 [[155 207 224]
  [155 207 224]
  [156 208 225]
  ...
  [130 147 168]
  [125 142 161]
  [119 134 153]]

 [[152 206 223]
  [152 206 223]
  [155 207 224]
  ...
  [108 127 148]
  [100 117 138]
  [ 94 108 127]]

 [[152 205 225]
  [149 205 222]
  [152 206 223]
  ...
  [ 86 104 121]
  [ 76  91 110]
  [ 65  80  99]]]
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ..

[[[253 242 250]
  [243 229 240]
  [213 187 205]
  ...
  [ 39  55 114]
  [ 32  57 119]
  [ 34  63 124]]

 [[222 177 196]
  [204 160 183]
  [174 123 151]
  ...
  [ 37  51 109]
  [ 40  60 118]
  [ 41  65 125]]

 [[161  89 119]
  [150  78 114]
  [136  59  97]
  ...
  [ 36  47 107]
  [ 46  60 126]
  [ 37  56 123]]

 ...

 [[ 85 208 128]
  [ 93 205 127]
  [ 93 202 123]
  ...
  [253 255 255]
  [253 255 255]
  [253 255 255]]

 [[ 92 213 132]
  [ 92 202 124]
  [ 96 202 125]
  ...
  [253 255 255]
  [253 255 255]
  [253 255 255]]

 [[ 93 213 132]
  [ 92 198 121]
  [ 88 195 116]
  ...
  [253 255 255]
  [253 255 255]
  [253 255 255]]]
[[[161 159 165]
  [160 158 164]
  [156 155 159]
  ...
  [249 244 241]
  [249 244 241]
  [249 244 241]]

 [[224 211 213]
  [210 197 199]
  [188 177 179]
  ...
  [249 244 241]
  [249 244 241]
  [249 244 241]]

 [[250 239 235]
  [252 241 237]
  [251 240 236]
  ...
  [249 244 241]
  [250 245 242]
  [250 245 242]]

 ...

 [[ 31  19  17]
  [ 30  18  16]
  [ 31  19  17]
  ..

[[[144 166 177]
  [144 166 177]
  [145 167 178]
  ...
  [101 111 111]
  [101 111 111]
  [101 111 111]]

 [[141 163 174]
  [141 163 174]
  [142 164 175]
  ...
  [101 111 111]
  [101 111 111]
  [101 111 111]]

 [[136 158 169]
  [136 158 169]
  [137 159 170]
  ...
  [101 111 111]
  [101 111 111]
  [101 111 111]]

 ...

 [[ 36  33  29]
  [ 31  28  24]
  [ 23  20  16]
  ...
  [230 225 224]
  [231 225 226]
  [230 225 226]]

 [[ 34  31  27]
  [ 29  26  22]
  [ 22  19  15]
  ...
  [227 222 221]
  [227 222 223]
  [227 222 223]]

 [[ 30  28  27]
  [ 25  23  22]
  [ 17  15  14]
  ...
  [225 220 219]
  [224 219 220]
  [224 219 220]]]
[[[144 166 177]
  [144 166 177]
  [145 167 178]
  ...
  [101 111 111]
  [101 111 111]
  [101 111 111]]

 [[141 163 174]
  [141 163 174]
  [142 164 175]
  ...
  [101 111 111]
  [101 111 111]
  [101 111 111]]

 [[136 158 169]
  [136 158 169]
  [137 159 170]
  ...
  [101 111 111]
  [101 111 111]
  [101 111 111]]

 ...

 [[ 36  33  29]
  [ 31  28  24]
  [ 23  20  16]
  ..

...

[[[185 183 189]
  [185 183 189]
  [184 182 188]
  ...
  [176 178 179]
  [177 178 182]
  [177 178 182]]

 [[185 183 189]
  [185 183 189]
  [185 183 189]
  ...
  [178 180 181]
  [177 179 180]
  [175 177 178]]

 [[186 184 190]
  [186 184 190]
  [186 184 190]
  ...
  [178 180 180]
  [177 179 180]
  [175 177 178]]

 ...

 [[181 180 182]
  [181 180 182]
  [184 180 185]
  ...
  [173 173 173]
  [174 173 175]
  [174 173 175]]

 [[181 180 182]
  [181 180 182]
  [184 180 185]
  ...
  [173 173 173]
  [174 173 175]
  [175 174 176]]

 [[183 180 182]
  [184 181 183]
  [185 181 186]
  ...
  [173 172 174]
  [174 173 177]
  [175 174 178]]]
[[[ 9 18 21]
  [ 9 18 21]
  [ 9 18 21]
  ...
  [ 0 15 18]
  [ 4 19 22]
  [ 9 24 27]]

 [[ 9 18 21]
  [ 9 18 21]
  [ 9 18 21]
  ...
  [ 0 15 18]
  [ 3 18 21]
  [ 5 20 23]]

 [[ 9 18 21]
  [ 9 18 21]
  [ 9 18 21]
  ...
  [ 0 15 18]
  [ 1 16 19]
  [ 2 17 20]]

 ...

 [[ 4 16 18]
  [ 5 17 19]
  [ 5 17 19]
  ...
  [18 14 20]
  [19 15 21]
  [17 13 19]]

 [[ 7 16 19]
  [ 8 1

[[[126 146 181]
  [126 146 181]
  [128 147 182]
  ...
  [127 150 188]
  [124 150 187]
  [125 151 188]]

 [[126 146 181]
  [126 146 181]
  [126 146 181]
  ...
  [125 149 185]
  [124 150 186]
  [126 152 188]]

 [[126 146 181]
  [127 147 182]
  [125 145 180]
  ...
  [123 149 185]
  [124 150 186]
  [125 151 187]]

 ...

 [[ 54  34  29]
  [ 52  32  27]
  [ 55  33  28]
  ...
  [ 48  91 140]
  [ 49  92 141]
  [ 50  93 142]]

 [[ 52  33  28]
  [ 51  32  27]
  [ 52  32  27]
  ...
  [ 47  90 139]
  [ 51  92 141]
  [ 52  93 142]]

 [[ 51  33  26]
  [ 51  33  26]
  [ 52  32  27]
  ...
  [ 46  89 138]
  [ 51  92 141]
  [ 52  93 142]]]
[[[116 115 117]
  [118 117 119]
  [118 117 119]
  ...
  [ 98  95  97]
  [ 96  93  95]
  [ 95  92  94]]

 [[117 116 118]
  [120 119 121]
  [123 122 124]
  ...
  [ 98  95  97]
  [ 97  94  96]
  [ 96  93  95]]

 [[115 114 116]
  [119 118 120]
  [124 123 125]
  ...
  [ 98  95  97]
  [ 97  94  96]
  [ 97  94  96]]

 ...

 [[ 54  58  63]
  [ 52  56  61]
  [ 52  55  60]
  ..

[[[209 206 208]
  [211 208 210]
  [212 209 211]
  ...
  [ 34  55 206]
  [ 34  55 206]
  [ 34  55 206]]

 [[209 207 207]
  [211 209 209]
  [211 209 209]
  ...
  [ 33  54 205]
  [ 33  54 205]
  [ 34  55 206]]

 [[209 207 207]
  [211 209 209]
  [211 209 209]
  ...
  [ 31  52 203]
  [ 32  53 204]
  [ 33  54 205]]

 ...

 [[223 228 227]
  [222 227 226]
  [222 227 225]
  ...
  [212 212 212]
  [212 212 212]
  [211 211 211]]

 [[222 228 227]
  [221 227 226]
  [222 227 226]
  ...
  [212 212 212]
  [212 212 212]
  [211 211 211]]

 [[222 228 227]
  [221 227 226]
  [221 227 226]
  ...
  [212 212 212]
  [212 212 212]
  [212 212 212]]]
[[[209 206 208]
  [211 208 210]
  [212 209 211]
  ...
  [ 34  55 206]
  [ 34  55 206]
  [ 34  55 206]]

 [[209 207 207]
  [211 209 209]
  [211 209 209]
  ...
  [ 33  54 205]
  [ 33  54 205]
  [ 34  55 206]]

 [[209 207 207]
  [211 209 209]
  [211 209 209]
  ...
  [ 31  52 203]
  [ 32  53 204]
  [ 33  54 205]]

 ...

 [[223 228 227]
  [222 227 226]
  [222 227 225]
  ..

[[[201 255 255]
  [201 255 255]
  [201 255 255]
  ...
  [201 255 255]
  [201 255 255]
  [201 255 255]]

 [[201 255 255]
  [201 255 255]
  [201 255 255]
  ...
  [201 255 255]
  [201 255 255]
  [201 255 255]]

 [[201 255 255]
  [201 255 255]
  [201 255 255]
  ...
  [201 255 255]
  [201 255 255]
  [201 255 255]]

 ...

 [[201 255 255]
  [201 255 255]
  [201 255 255]
  ...
  [201 255 255]
  [201 255 255]
  [201 255 255]]

 [[201 255 255]
  [201 255 255]
  [201 255 255]
  ...
  [201 255 255]
  [201 255 255]
  [201 255 255]]

 [[201 255 255]
  [201 255 255]
  [201 255 255]
  ...
  [201 255 255]
  [201 255 255]
  [201 255 255]]]
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [  1 124 150]
  [  0 122 148]
  [  0 121 146]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [  9 130 156]
  [  6 129 155]
  [  2 128 153]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [ 11 132 158]
  [  8 131 157]
  [  6 129 155]]

 ...

 [[  0   1  35]
  [  0   1  35]
  [  0   1  35]
  ..

[[[ 42  80  68]
  [ 40  77  67]
  [ 39  79  68]
  ...
  [ 56  74  81]
  [ 55  73  80]
  [ 56  74  81]]

 [[ 42  80  68]
  [ 40  80  68]
  [ 40  80  69]
  ...
  [ 57  75  82]
  [ 56  74  81]
  [ 56  74  81]]

 [[ 40  80  68]
  [ 42  82  70]
  [ 42  82  70]
  ...
  [ 54  72  79]
  [ 56  74  81]
  [ 58  77  82]]

 ...

 [[ 79 118 127]
  [ 79 118 127]
  [ 79 118 127]
  ...
  [ 64  79  88]
  [ 54  71  80]
  [ 55  71  83]]

 [[ 82 118 128]
  [ 79 118 127]
  [ 78 117 125]
  ...
  [ 70  85  94]
  [ 61  78  87]
  [ 54  70  82]]

 [[ 83 119 129]
  [ 80 117 125]
  [ 80 117 125]
  ...
  [ 71  86  95]
  [ 57  74  83]
  [ 57  73  85]]]
[[[ 78  83  82]
  [ 80  85  84]
  [ 82  87  88]
  ...
  [142 155 153]
  [141 154 152]
  [140 153 151]]

 [[ 80  85  84]
  [ 82  87  86]
  [ 84  89  90]
  ...
  [142 155 153]
  [141 154 152]
  [140 153 151]]

 [[ 82  88  87]
  [ 84  90  89]
  [ 87  93  92]
  ...
  [143 156 154]
  [142 155 153]
  [141 154 152]]

 ...

 [[138 123 120]
  [140 125 122]
  [143 128 125]
  ..

In [ ]:
# In the same way write annotation for test.csv
"""
test_df = pd.read_csv('test.csv')

# for test
with open("test_annotation.txt", "w+") as f:
  for idx, row in test_df.iterrows():
      sys.stdout.write(str(idx) + '\r')
      sys.stdout.flush()
      img = cv2.imread('test/' + row['ImgID'])
      height, width = img.shape[:2]
      x1 = int(row['XMin'] * width)
      x2 = int(row['XMax'] * width)
      y1 = int(row['YMin'] * height)
      y2 = int(row['YMax'] * height)
      
      test_file_path = 'D:/5-deep_learning_with_tensorflow/module 7 cnn/7-faster_r-cnn/Faster_r-cnn_demo_youtube/test'
      fileName = os.path.join(test_file_path, row['FileName'])
      className = row['ClassName']
      f.write(fileName + ',' + str(x1) + ',' + str(y1) + ',' + str(x2) + ',' + str(y2) + ',' + className + '\n')
      ""